# Cost overview

- parse usage output and calculate costs for an example

- [ ] Check prices for CR2

In [ ]:
import pprint
import pathlib

import yaml
import pandas as pd

from src.parser import parse_usage_output

TESTFILE = pathlib.Path('logs/') / 'usage_2022_02_10.txt'

usage = parse_usage_output(TESTFILE)
print(*usage[:-1], sep='\n') 
pprint.pprint(usage.data)

In [ ]:
storage_gb = int(usage.storage.split(':')[1].split()[0])
storage_gb

In [ ]:
# {key.strip(): float(value) for key, value in zip(headers, total[1:])}
_total = {}
for key, value in zip(usage.headers, usage.total[1:]):
    try:
        _total[key.strip()] = int(value)
    except ValueError:
        _total[key.strip()] = float(value)
usage.data[usage.total[0].strip(':')] = _total
data = usage.data


In [ ]:
df = pd.DataFrame(data).T
df

In [ ]:
with open('cr2_prices.yaml') as f:
    prices = yaml.safe_load(f)
prices

In [ ]:
print(f"Storage costs: {prices['storage_TB']*storage_gb/1024:.2f} DKK")

In [ ]:
costs = df[usage.headers[0]] * prices['thin_node'] / 40
costs = costs.to_frame('thin nodes')
costs['fat nodes'] = df[usage.headers[2]] * prices['fat_node'] / 40
costs['gpu nodes'] = df[usage.headers[4]] * prices['gpu_node'] / 40 
costs['total'] = costs.sum(axis=1)
costs.columns.name = 'in DKK:'
costs.index.name = 'user:'
costs

In [ ]:
msg = f"```\n{costs.to_string(float_format='{:.1f}'.format)}\n```"
print(msg)